# Anthropic Claude Requirements Discovery
**Discovers and validates requirement associations using Claude AI models with configurable similarity thresholds and association probability filtering.**


In [ ]:
# Cell [0] - Setup and Imports
# Purpose: Import all required libraries and configure environment settings for Multi-LLM testing
# Dependencies: os, sys, json, pathlib, dotenv, datetime, pandas, praxis_sentence_transformer
# Breadcrumbs: Setup -> Imports -> Environment Configuration

import os
import sys
import json
from pathlib import Path
from dotenv import load_dotenv
from datetime import datetime
import pandas as pd
import logging
import matplotlib.pyplot as plt

# Import from praxis_sentence_transformer package (installed via pip)
from praxis_sentence_transformer.neo4j_operations import Neo4jClient
from praxis_sentence_transformer.analysis.analyzer import RequirementsAnalyzer
from praxis_sentence_transformer.clients.claude import ClaudeRequirementAnalyzer
from praxis_sentence_transformer.logger import setup_logging, DebugTimer
from praxis_sentence_transformer.visualization import RequirementsVisualizer

# Set up logging
logger = setup_logging("neo4j-notebook", logging.DEBUG)

# Load environment variables
load_dotenv()

In [ ]:
# Cell [1] - Neo4j Client Initialization
# Purpose: Initialize Neo4j client and establish database connection for requirement data access
# Dependencies: Neo4jClient, logger, sys
# Breadcrumbs: Setup -> Database Connection -> Client Initialization

neo4j_client = Neo4jClient()

# Test connection
if not neo4j_client.connect():
    logger.error("Failed to connect to Neo4j database")
    sys.exit(1)

In [ ]:
# Cell [2] - Configuration and Parameter Setup
# Purpose: Configure model parameters and create output directories for analysis results
# Dependencies: Path, logger
# Breadcrumbs: Database Connection -> Configuration -> Parameter Definition

# Model configuration
model_name = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
alpha = 0.6
threshold = 0.4

# Create results directory
results_dir = Path("results/neo4j_analysis")
results_dir.mkdir(parents=True, exist_ok=True)

logger.info(f"Using model: {model_name}")
logger.info(f"Alpha: {alpha}")
logger.info(f"Threshold: {threshold}")

In [ ]:
# Cell [3] - Requirement Mappings Retrieval
# Purpose: Retrieve all requirement mappings from Neo4j database using configured parameters
# Dependencies: neo4j_client, json, logger
# Breadcrumbs: Configuration -> Data Retrieval -> Mapping Extraction

# Get all mappings in one go
mappings = neo4j_client.get_all_requirement_mappings(
    model_name="sentence-transformers/multi-qa-mpnet-base-dot-v1",
    alpha=0.6,
    threshold=0.4
)

# Log summary statistics at INFO level
logger.info(f"Total source requirements: {len(mappings['mappings'])}")
logger.info(f"Total relationships: {mappings['metadata']['total_relationships']}")

# Log detailed mapping data at DEBUG level
for mapping in mappings['mappings'][:5]:  # First 5 for example
    source = mapping['source']
    targets = mapping['targets']
    logger.info(f"Source {source['id']} has {len(targets)} matches")
    # Optional: Add more detailed debug logging for each source's targets
    for target in targets[:3]:  # Show first 3 targets for each source
        logger.info(f"  - Target: {target['id']}, Similarity: {target['similarity']:.3f}")
        
# Create and log first 3 requirements with only first 3 targets each
first_three = {
    "metadata": mappings["metadata"],
    "mappings": [
        {
            "source": mapping["source"],
            "targets": mapping["targets"][:3]  # Limit to first 3 targets
        }
        for mapping in mappings["mappings"][:3]  # Limit to first 3 mappings
    ]
}


logger.debug(f"\nFirst 3 requirements structure:\n{json.dumps(first_three, indent=2)}")

In [ ]:
# Cell [4] - Claude Client Setup and Mapping Processing
# Purpose: Initialize Claude AI analyzer and process requirement mappings for association analysis
# Dependencies: os, ClaudeRequirementAnalyzer, DebugTimer, logger
# Breadcrumbs: Data Retrieval -> AI Analysis -> Claude Processing Pipeline

# Get configuration from environment
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
claude_model = os.getenv("CLAUDE_3_5_MODEL")
min_association_prob = float(os.getenv("MIN_ASSOCIATION_PROBABILITY", 0.6))

# Validate environment variables
if not anthropic_api_key:
    raise ValueError("ANTHROPIC_API_KEY not found in environment variables")
if not claude_model:
    raise ValueError("CLAUDE_3_5_MODEL not found in environment variables")

logger.debug("Retrieved configuration from environment variables")

# Initialize Claude analyzer
try:
    claude_analyzer = ClaudeRequirementAnalyzer(
        api_key=anthropic_api_key,
        model_name=claude_model,
        min_association_probability=min_association_prob
    )
    logger.info(f"Successfully initialized Claude analyzer with model: {claude_model}")
    
    # Log the system prompt being used
    logger.debug(f"Using system prompt:\n{claude_analyzer.system_prompt}")
    
    # Process mappings with Claude
    logger.info("Starting mapping processing with Claude...")
    timer = DebugTimer("Processing mappings with Claude")
    timer.start()
    
    claude_requirements_results_set = claude_analyzer.process_mappings(mappings)
    
    timer.stop()
    logger.info(f"Processing completed in {timer.duration:.2f} seconds")
    
    # Log summary statistics
    logger.info(f"Processed {claude_requirements_results_set.total_target_matches} requirement matches")
    logger.info(f"Found {claude_requirements_results_set.total_associated_matches} associated matches")
    
    # Show detailed debug info about some matches
    if claude_requirements_results_set.processed_matches:
        logger.info("Sample of processed matches:")
        for i, match in enumerate(claude_requirements_results_set.processed_matches[:2]):
            logger.debug(f"""
            Match {i+1}:
            Source ID: {match.source_id}
            Source Content: {match.source_content[:100]}...
            Target ID: {match.target_id}
            Target Content: {match.target_content[:100]}...
            Similarity Score: {match.similarity_score:.3f}
            Association Probability: {match.association_probability:.3f}
            Is Associated: {match.is_associated}
            Explanation: {match.explanation}
            """)
        # Print summary of results
        logger.info("\nClaude Analysis Summary:")
        logger.info(f"Total source requirements processed: {claude_requirements_results_set.total_source_requirements}")
        logger.info(f"Total target matches analyzed: {claude_requirements_results_set.total_target_matches}")
        logger.info(f"Total associated matches found: {claude_requirements_results_set.total_associated_matches}")

        # Print breakdown by source requirement
        logger.info("\nBreakdown by source requirement:")
        source_counts = {}
        associated_counts = {}

        for match in claude_requirements_results_set.processed_matches:
            source_counts[match.source_id] = source_counts.get(match.source_id, 0) + 1
            if match.is_associated:
                associated_counts[match.source_id] = associated_counts.get(match.source_id, 0) + 1

        for source_id, count in source_counts.items():
            associated = associated_counts.get(source_id, 0)
            logger.info(f"Source {source_id}:")
            logger.info(f"  - Total target matches: {count}")
            logger.info(f"  - Associated matches: {associated}")

except Exception as e:
    logger.error(f"Error in Claude analysis: {str(e)}")
    raise

In [ ]:
# Cell [5] - LLM Requirement Traces Creation
# Purpose: Create and store LLM requirement traces in Neo4j database for analysis tracking
# Dependencies: neo4j_client, logger
# Breadcrumbs: Claude Processing -> Database Storage -> Trace Creation

if claude_requirements_results_set.processed_matches:
    # Configuration
    sentence_transformer_model = model_name
    llm_model = claude_model  # Using the Claude model name from environment

    logger.info("Creating LLM requirement traces...")
    logger.info(f"Sentence Transformer Model: {sentence_transformer_model}")
    logger.info(f"LLM Model: {llm_model}")

    # Create traces
    success_count, failure_count = neo4j_client.create_llm_traces_from_results(
        results_set=claude_requirements_results_set,
        model_name=sentence_transformer_model,
        llm_model_name=llm_model
    )

    # Log results
    logger.info(f"Successfully created {success_count} LLM requirement traces")
    if failure_count > 0:
        logger.warning(f"Failed to create {failure_count} traces")

    # Verify creation
    verification_query = f"""
    MATCH ()-[r:LLM_REQUIREMENT_TRACE]->()
    WHERE r.llm_model_name = '{llm_model}'
    RETURN count(r) as count
    """

    with neo4j_client.driver.session(database=neo4j_client.database) as session:
        result = session.run(verification_query)
        trace_count = result.single()["count"]
        logger.info(f"Total LLM requirement traces in database for {llm_model}: {trace_count}")

In [ ]:
# Cell [6] - Statistical Analysis and Metrics Calculation
# Purpose: Perform comprehensive statistical analysis and calculate performance metrics
# Dependencies: RequirementsAnalyzer, pandas, json, datetime, logger
# Breadcrumbs: Trace Creation -> Performance Analysis -> Statistical Evaluation

logger.info("Performing statistical analysis of results")

# Initialize analyzer if not already initialized
analyzer = RequirementsAnalyzer(
    neo4j_client=neo4j_client,
    sentence_transformer_model=model_name,
    llm_model_name=claude_model,
    alpha=alpha,
    threshold=threshold
)

# Calculate basic metrics
metrics = analyzer.calculate_metrics()

# Print detailed results
logger.info("\nDetailed Metrics:")
logger.info(f"True Positives: {metrics['true_positives']}")
logger.info(f"False Positives: {metrics['false_positives']}")
logger.info(f"True Negatives: {metrics['true_negatives']}")
logger.info(f"False Negatives: {metrics['false_negatives']}")
logger.info(f"\nPrecision: {metrics['precision']:.4f}")
logger.info(f"Recall: {metrics['recall']:.4f}")
logger.info(f"Accuracy: {metrics['accuracy']:.4f}")
logger.info(f"Balanced Accuracy: {metrics['balanced_accuracy']:.4f}")
logger.info(f"F1 Score: {metrics['f1_score']:.4f}")

# Get confusion matrix
conf_matrix = analyzer.get_confusion_matrix()
logger.info("\nConfusion Matrix:")
logger.info("[[TN, FP]")
logger.info(" [FN, TP]]")
logger.info(f"\n{conf_matrix}")

# Get detailed classification information
logger.info("\nDetailed Classification Analysis:")
classification_details = analyzer.get_all_classification_details()

# True Positives Analysis
tp_details = pd.DataFrame(classification_details['true_positives'])
if not tp_details.empty:
    logger.info("\nTrue Positives Details:")
    logger.info(f"Total Count: {len(tp_details)}")
    logger.info("\nSample of True Positive matches:")
    logger.info(tp_details.head().to_string())
    logger.info(f"\nAverage LLM Confidence: {tp_details['llm_confidence'].mean():.4f}")
    logger.info(f"Average Ground Truth Confidence: {tp_details['ground_truth_confidence'].mean():.4f}")

# False Positives Analysis
fp_details = pd.DataFrame(classification_details['false_positives'])
if not fp_details.empty:
    logger.info("\nFalse Positives Details:")
    logger.info(f"Total Count: {len(fp_details)}")
    logger.info("\nSample of False Positive matches:")
    logger.info(fp_details.head().to_string())
    logger.info(f"\nAverage LLM Confidence: {fp_details['llm_confidence'].mean():.4f}")

# False Negatives Analysis
fn_details = pd.DataFrame(classification_details['false_negatives'])
if not fn_details.empty:
    logger.info("\nFalse Negatives Details:")
    logger.info(f"Total Count: {len(fn_details)}")
    logger.info("\nSample of False Negative matches:")
    logger.info(fn_details.head().to_string())
    logger.info(f"\nAverage Ground Truth Confidence: {fn_details['ground_truth_confidence'].mean():.4f}")

# True Negatives Analysis
tn_details = pd.DataFrame(classification_details['true_negatives'])
if not tn_details.empty:
    logger.info("\nTrue Negatives Details:")
    logger.info(f"Total Count: {len(tn_details)}")
    logger.info("\nSample of True Negative pairs:")
    logger.info(tn_details.head().to_string())

# Save detailed results to file
results_dir = Path("results/analysis")
results_dir.mkdir(parents=True, exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
results_file = results_dir / f"detailed_analysis_{timestamp}.json"

detailed_results = {
    'metrics': metrics,
    'confusion_matrix': conf_matrix.tolist(),
    'classification_details': classification_details
}

with open(results_file, 'w') as f:
    json.dump(detailed_results, f, indent=2)

logger.info(f"\nDetailed analysis results saved to {results_file}")

In [ ]:
# Cell [7] - Results Visualization and Plotting
# Purpose: Create comprehensive visualizations for analysis results and performance metrics
# Dependencies: RequirementsVisualizer, Path, datetime, pandas, matplotlib
# Breadcrumbs: Statistical Evaluation -> Visualization -> Results Presentation

# Initialize visualizer with model information
visualizer = RequirementsVisualizer(
    figsize=(8, 6),  # Reduced figure size
    st_model=model_name,
    llm_model=claude_model
)

# Get current timestamp for filenames
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Create model-specific results directory
st_name = model_name.replace('/', '_').replace('-', '_')
llm_name = claude_model.replace('-', '_')
results_dir = Path(f"results/analysis_plots/{llm_name}/{st_name}")
results_dir.mkdir(parents=True, exist_ok=True)

# Get metrics for Claude model
metrics = analyzer.calculate_metrics()

# Plot and display confusion matrix
plt.figure(figsize=(8, 6))  # Controlled figure size
conf_matrix = analyzer.get_confusion_matrix()
visualizer.plot_confusion_matrix(
    conf_matrix,
    title="Model Confusion Matrix",
    save_path=str(results_dir / f"confusion_{timestamp}.png"),
    dpi=100
)
plt.show()

# Create and display metrics summary table
metrics_to_display = [
    'precision', 'recall', 'f1_score', 
    'accuracy', 'balanced_accuracy'
]

metrics_df = pd.DataFrame({
    'Metric': metrics_to_display,
    'Value': [f"{metrics[metric]:.4f}" for metric in metrics_to_display]
})

# Set pandas display options for better formatting
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Display the metrics table
display(metrics_df)

# Log paths to saved plots
logger.info(f"\nPlots saved to:")
logger.info(f"- Confusion Matrix: {results_dir}/confusion_{timestamp}.png")

# Close any remaining matplotlib figures
plt.close('all')

In [ ]:
# Cell [8] - Cleanup and Session Termination
# Purpose: Close database connections and finalize the analysis session
# Dependencies: neo4j_client, logger
# Breadcrumbs: Results Presentation -> Session Management -> Resource Cleanup

neo4j_client.close()
logger.info("Analysis complete")